# TorchSDE + Neuromancer: Latent Stochastic Differential Equations (System ID of Stochastic Process)

This notebook goes over how to utilize torchsde's functionality within Neuromancer framework. This notebook is based off: https://github.com/google-research/torchsde/blob/master/examples/latent_sde_lorenz.py. In this example, we generate data according to a 3-dimensional stochastic Lorenz attractor. We then perform a "system identification" on this data -- seek to model a stochastic differential equation on this data. Upon performant training, this LatentSDE will then be able to reproduce samples that exhibit the same behavior as the provided Lorenz system. We train and utilize the Lightning framework to support custom functionality within the training loop.




### Imports

In [3]:
import torch
import os

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
import torch
import tqdm
from neuromancer.psl import plot
from neuromancer import psl
import torchsde
import torchsde

from torch.utils.data import DataLoader
from neuromancer.system import Node
from neuromancer.dynamics import integrators, ode
from neuromancer.trainer import Trainer, LitTrainer
from neuromancer.problem import Problem
from neuromancer.loggers import BasicLogger
from neuromancer.dataset import DictDataset
from neuromancer.constraint import variable
from neuromancer.loss import PenaltyLoss
from neuromancer.modules import blocks
from torch import nn
from torch import optim
from torch.distributions import Normal
from typing import Sequence

torch.manual_seed(0)


### Functions to generate data from a Lorenz attractor

In [4]:
class LinearScheduler(object):
    def __init__(self, iters, maxval=1.0):
        self._iters = max(1, iters)
        self._val = maxval / self._iters
        self._maxval = maxval

    def step(self):
        self._val = min(self._maxval, self._val + self._maxval / self._iters)

    @property
    def val(self):
        return self._val


class StochasticLorenz(object):
    """Stochastic Lorenz attractor.

    Used for simulating ground truth and obtaining noisy data.
    Details described in Section 7.2 https://arxiv.org/pdf/2001.01328.pdf
    Default a, b from https://openreview.net/pdf?id=HkzRQhR9YX
    """
    noise_type = "diagonal"
    sde_type = "ito"

    def __init__(self, a: Sequence = (10., 28., 8 / 3), b: Sequence = (.1, .28, .3)):
        super(StochasticLorenz, self).__init__()
        self.a = a
        self.b = b

    def f(self, t, y):
        x1, x2, x3 = torch.split(y, split_size_or_sections=(1, 1, 1), dim=1)
        a1, a2, a3 = self.a

        f1 = a1 * (x2 - x1)
        f2 = a2 * x1 - x2 - x1 * x3
        f3 = x1 * x2 - a3 * x3
        return torch.cat([f1, f2, f3], dim=1)

    def g(self, t, y):
        x1, x2, x3 = torch.split(y, split_size_or_sections=(1, 1, 1), dim=1)
        b1, b2, b3 = self.b

        g1 = x1 * b1
        g2 = x2 * b2
        g3 = x3 * b3
        return torch.cat([g1, g2, g3], dim=1)

    @torch.no_grad()
    def sample(self, x0, ts, noise_std, normalize):
        """Sample data for training. Store data normalization constants if necessary."""
        xs = torchsde.sdeint(self, x0, ts)
        if normalize:
            mean, std = torch.mean(xs, dim=(0, 1)), torch.std(xs, dim=(0, 1))
            xs.sub_(mean).div_(std).add_(torch.randn_like(xs) * noise_std)
        return xs


## Neuromancer Integration

As per the NeuroMANCER x Lightning workflow, generate the data_setup_function and return the DictDatasets. Note that we only have a train dataset here, so we return `None` for dev/test datasets

In [6]:
def make_dataset(t0, t1, batch_size, noise_std, steps=100):
    _y0 = torch.randn(batch_size, 3)
    ts = torch.linspace(t0, t1, steps=steps)
    xs = StochasticLorenz().sample(_y0, ts, noise_std, normalize=True)
    train_data = DictDataset({'xs':xs},name='train')
    return train_data, None, None, batch_size
    

Define some experimental parameters

In [7]:
batch_size=256
latent_size=4
context_size=64
hidden_size=128
lr_init=1e-2
t0=0.
t1=2.
lr_gamma=0.997
num_iters=1
kl_anneal_iters=1000
pause_every=50
noise_std=0.01
method="euler"
steps = 100 # number of time steps

Define Neuromancer components, variables, and problem to train the LatentSDE. Upon training, this LatentSDE will generate new samples that exhibit the behavior of the Lorenz attractor training data. For this example, we set `adjoint` to `False` (do not use the adjoint sensitivity method). This is because this method seems to be significantly slower. 

Also note that we need to pass in the timestep tensor to our `LatentSDE_Encoder`, and as a result need to also define it outside the `make_dataset()` function. We note that this is not the cleanest code and breaks the data abstraction. Additional features will be added to mitigate this.

In [8]:
ts = torch.linspace(t0, t1, steps=steps)

sde_block_encoder = blocks.LatentSDE_Encoder(3, latent_size, context_size, hidden_size, ts=ts, adjoint=False) 
integrator = integrators.LatentSDEIntegrator(sde_block_encoder, adjoint=False)
model_1 = Node(integrator, input_keys=['xs'], output_keys=['zs', 'z0', 'log_ratio',  'xs', 'qz0_mean', 'qz0_logstd'], name='m1')
sde_block_decoder = blocks.LatentSDE_Decoder(3, latent_size, noise_std=noise_std)
model_2 = Node(sde_block_decoder, input_keys=['xs', 'zs', 'log_ratio', 'qz0_mean', 'qz0_logstd'], output_keys=['xs_hat', 'log_pxs', 'sum_term', 'log_ratio'], name='m2' )

xs = variable('xs')
zs = variable('zs')
z0 = variable('z0')
xs_hat = variable('xs_hat')


log_ratio = variable('log_ratio')
qz0_mean = variable('qz0_mean')
qz0_logstd = variable('qz0_logstd')
log_pxs = variable('log_pxs')
sum_term = variable('sum_term')

# NeuroMANCER loss function format
loss = (-1.0*log_pxs + log_ratio) == 0.0

# aggregate list of objective terms and constraints
objectives = [loss]
constraints = []
# create constrained optimization loss
loss = PenaltyLoss(objectives, constraints)
# construct constrained optimization problem
problem = Problem([model_1, model_2], loss)

We now define helper visualization function (again see https://github.com/google-research/torchsde/blob/master/examples/latent_sde_lorenz.py) that will fire every N epochs in our training loop. This visualization will allow us to see the learned Lorenz attractor samples 

In [9]:
# Fix the same Brownian motion for visualization.
bm_vis = torchsde.BrownianInterval(
    t0=t0, t1=t1, size=(batch_size, latent_size,), device='cpu', levy_area_approximation="space-time")

# 
def vis(data_dict, problem, bm_vis, img_path, num_samples=10):
    encoder, decoder = problem.nodes[0], problem.nodes[1] #extract the encoder and decoder from our problem

    fig = plt.figure(figsize=(20, 9))
    gs = gridspec.GridSpec(1, 2)
    ax00 = fig.add_subplot(gs[0, 0], projection='3d')
    ax01 = fig.add_subplot(gs[0, 1], projection='3d')

    xs = data_dict['xs'] #pull out data sample from the DictDataset
    # Left plot: data.
    z1, z2, z3 = np.split(xs.cpu().numpy(), indices_or_sections=3, axis=-1)
    [ax00.plot(z1[:, i, 0], z2[:, i, 0], z3[:, i, 0]) for i in range(num_samples)]
    ax00.scatter(z1[0, :num_samples, 0], z2[0, :num_samples, 0], z3[0, :10, 0], marker='x')
    ax00.set_yticklabels([])
    ax00.set_xticklabels([])
    ax00.set_zticklabels([])
    ax00.set_xlabel('$z_1$', labelpad=0., fontsize=16)
    ax00.set_ylabel('$z_2$', labelpad=.5, fontsize=16)
    ax00.set_zlabel('$z_3$', labelpad=0., horizontalalignment='center', fontsize=16)
    ax00.set_title('Data', fontsize=20)
    xlim = ax00.get_xlim()
    ylim = ax00.get_ylim()
    zlim = ax00.get_zlim()

    # Right plot: samples from learned model.
    mydata = data_dict
    output = decoder(encoder(mydata))
    xs_hat = output['xs_hat'].detach().cpu().numpy() 
    #xs = latent_sde.sample(batch_size=xs.size(1), ts=ts, bm=bm_vis).cpu().numpy()
    z1, z2, z3 = np.split(xs_hat, indices_or_sections=3, axis=-1)

    [ax01.plot(z1[:, i, 0], z2[:, i, 0], z3[:, i, 0]) for i in range(num_samples)]
    ax01.scatter(z1[0, :num_samples, 0], z2[0, :num_samples, 0], z3[0, :10, 0], marker='x')
    ax01.set_yticklabels([])
    ax01.set_xticklabels([])
    ax01.set_zticklabels([])
    ax01.set_xlabel('$z_1$', labelpad=0., fontsize=16)
    ax01.set_ylabel('$z_2$', labelpad=.5, fontsize=16)
    ax01.set_zlabel('$z_3$', labelpad=0., horizontalalignment='center', fontsize=16)
    ax01.set_title('Samples', fontsize=20)
    ax01.set_xlim(xlim)
    ax01.set_ylim(ylim)
    ax01.set_zlim(zlim)

    plt.savefig(img_path)
    plt.close()

### Neuromancer training the problem to learn the stochastic process

We now train and visualize results using Lightning workflow

In [ ]:

# Fix the same Brownian motion for visualization.
bm_vis = torchsde.BrownianInterval(
    t0=t0, t1=t1, size=(batch_size, latent_size,), device='cpu', levy_area_approximation="space-time")

# Define the custom_training_step to support visualization. 
def custom_training_step(model, batch): 
    output = model.problem(batch)
    loss = output[model.train_metric]
    img_path = os.path.join('', f'current_epoch_{model.current_epoch:06d}.pdf')
    if model.current_epoch % 50 == 0: 
        vis(batch,  model.problem, bm_vis, img_path, num_samples=10)
    return loss


optimizer = torch.optim.Adam(problem.parameters(), lr=0.001)
lit_trainer = LitTrainer(epochs=300, accelerator='cpu', train_metric='train_loss', 
                         dev_metric='train_loss', eval_metric='train_loss', test_metric='train_loss',
                         custom_optimizer=optimizer, custom_training_step=custom_training_step)



lit_trainer.fit(problem=problem, data_setup_function=make_dataset,  t0=t0, t1=t1, batch_size=batch_size, noise_std=noise_std)
